In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


# read data
data =pd.read_csv('Encoded_Results_21Mar2022.csv')
# Standard normalization
columns_to_normalize=['mean_ghgs','mean_watscar','mean_bio','mean_eut','mean_watuse','mean_acid','sd_ghgs','sd_watscar','sd_bio','sd_eut','sd_watuse','sd_acid']
data_to_normalize=data[columns_to_normalize]
scaler = StandardScaler()
normalized_data = scaler.fit_transform(data_to_normalize)
normalized_df = pd.DataFrame(normalized_data, columns=columns_to_normalize, index=data.index)
normalized_df['age_group'] = data['age_group']
normalized_df['sex'] = data['sex']

# combined sex and age_group into a single feature
normalized_df['age_sex']=normalized_df['age_group']+'_'+normalized_df['sex']
grouped_data =normalized_df.groupby(['age_sex'])[columns_to_normalize].mean()

# export csv
grouped_data.to_csv('transfer_data.csv', index=True)

In [10]:
# read data
data = pd.read_csv('transfer_data.csv')

# converted to long-format data
mean_col=[col for col in data.columns if 'mean_' in col]
sd_col=[col for col in data.columns if 'sd_' in col]
mean_data=data.melt(id_vars=['age_sex'], value_vars=mean_col, var_name='Indicator', value_name='Value')
mean_data['Indicator']= mean_data['Indicator'].str.replace('mean_', '')
sd_data = data.melt(id_vars=['age_sex'], value_vars=sd_col, var_name='Indicator', value_name='Sd')
sd_data['Indicator'] = sd_data['Indicator'].str.replace('sd_', '')
final_data = pd.merge(mean_data, sd_data, on=['age_sex', 'Indicator'])

# export csv
final_data.to_csv('final_data.csv', index=False)